In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pymc3 as pm
import arviz as az

sns.set()

In [ ]:

nr_samples = 10
inside = np.array([23,19,23,19,23,19,23,19,23,19])
                                    
outside = np.array([-20,-20,-15,-15,-10,-10,-8,-8,-4,-4])

print (inside)
print (outside)

e = (2 * inside) ** 2 + outside ** 2 + inside * -outside
print (e)

In [ ]:
df = pd.DataFrame({'out': outside,
                  'in' : inside,
                  'e' : e})

dates = pd.date_range('2020-01-01',periods=len(df))
df.index = dates
df

In [ ]:
ax=df.plot(y='e',style='ro--',figsize=(18,12),
           title='Energy consuption given inside & outside temp')
ax2 = plt.twinx()

df[['in','out']].plot(ax=ax2,style=['go--','bo--'])

ax.legend(loc='upper left')
ax.set_ylabel('energy')

ax2.legend(loc='upper right')
ax2.set_ylabel('temp [inside,outside]')

In [ ]:
df.corr()

In [ ]:

def standardize(series):
    return (series - series.mean()) / series.std()

df[['out_std','in_std','e_std']] = df.apply(standardize)
df['day_idx'] = range(len(df))
df['country_idx'] = 0
df['isolation_idx'] = 0
df

In [ ]:
df2 = df.copy()
df2['country_idx'] = 1
df2['isolation_idx'] = 1
df2['e'] = df2['e'] * 1.1
df2['e_std'] = standardize(df2['e'])

In [ ]:
df2

In [ ]:
df_all = pd.concat([df,df2])
df_all

In [ ]:
plt.scatter(df['out_std'],df['e_std'])
plt.title('regression out -> energy, std units')
plt.xlabel('out')
plt.ylabel('energy')

In [ ]:
# model 1 single regression outside temp -> energy

x = df['out_std'].values

model1 = pm.Model()

result = pd.DataFrame()

with model1:
    alpha = pm.Normal('alpha',mu=0,sd=1)
    beta = pm.Normal('beta',mu=0,sd=1)
    obs_sigma = pm.Exponential('obs_sigma',1)
    
    reg = alpha + x * beta
    lkh = pm.Normal('lkh',mu=reg,sd=obs_sigma,observed=df['e_std'])
    
    trace = pm.sample(500,tune=500)
    
    result = pm.trace_to_dataframe(trace)
    
    
    

In [ ]:
result

In [ ]:
summary = pd.DataFrame()

with model1:
    az.plot_trace(trace)
    summary = az.summary(trace,hdi_prob=0.89)
    print (summary)

In [ ]:
with model1:

    print (az.summary(trace,hdi_prob=0.89))
    


In [ ]:
nr_rows = 1000

X = np.linspace(df['out_std'].min(),df['out_std'].max(),100)

rows = np.random.choice(result.index,replace=True,size=nr_rows)

lines = np.array([result.iloc[rows]['alpha'] + result.iloc[rows]['beta'] * X[i] for i in range(len(X))])



In [ ]:
_= plt.plot(X,lines,color='r',alpha=0.01)

ax = plt.gca()

label_1 = r'$\alpha$ : {:.2f} [HPDI : {:.2f} .. {:.2f}]'.format(summary.loc['alpha','mean'],
                                                                summary.loc['alpha','hdi_5.5%'],
                                                                summary.loc['alpha','hdi_94.5%'])

label_2 = r'$\beta$ : {:.2f} [HPDI : {:.2f} .. {:.2f}]'.format(summary.loc['beta','mean'],
                                                              summary.loc['beta','hdi_5.5%'],
                                                              summary.loc['beta','hdi_94.5%'])

_= plt.plot(X,result['alpha'].mean() + result['beta'].mean() * X,
            color='k',ls='dashed',label=label_1 + '\n' + label_2)

plt.xlabel('out [std]')
plt.ylabel('energy [std]')

plt.legend(loc='upper left')

fill_kwargs = {'alpha': 0.3}
az.plot_hpd(X, lines.T,ax=ax,color='c',fill_kwargs=fill_kwargs,hdi_prob=0.89)

plt.scatter(df['out_std'],df['e_std'],color='crimson')

In [ ]:
# model_2 indoor --> energy

x = df['in_std'].values

model_2 = pm.Model()
with model_2:
    
    alpha = pm.Normal('alpha',0,1)
    beta = pm.Normal('beta',0,1)
    obs_sigma = pm.Exponential('obs_sigma',1)
    
    reg = alpha + x * beta
    
    lkh = pm.Normal('lkh',reg,obs_sigma,observed=df['e_std'])
    
    trace = pm.sample(500,tune=500)

In [ ]:
summary = pd.DataFrame()
result = pd.DataFrame()


with model_2:
    result = pm.trace_to_dataframe(trace)
    az.plot_trace(trace)
    summary = az.summary(trace,hdi_prob=0.89)

summary

In [ ]:
result.describe()

In [ ]:
rows = np.random.choice(result.index,replace=True,size=nr_rows)

lines = np.array([result.iloc[rows]['alpha'] + result.iloc[rows]['beta'] * X[i] for i in range(len(X))])

In [ ]:
_= plt.plot(X,lines,color='r',alpha=0.02)

ax = plt.gca()

label_1 = r'$\alpha$ : {:.2f} [HPDI : {:.2f} .. {:.2f}]'.format(summary.loc['alpha','mean'],
                                                                summary.loc['alpha','hdi_5.5%'],
                                                                summary.loc['alpha','hdi_94.5%'])

label_2 = r'$\beta$ : {:.2f} [HPDI : {:.2f} .. {:.2f}]'.format(summary.loc['beta','mean'],
                                                              summary.loc['beta','hdi_5.5%'],
                                                              summary.loc['beta','hdi_94.5%'])

_= plt.plot(X,result['alpha'].mean() + result['beta'].mean() * X,
            color='k',ls='dashed',label=label_1 + '\n' + label_2)

plt.xlabel('in [std]')
plt.ylabel('energy [std]')

plt.legend(loc='upper left')

fill_kwargs = {'alpha': 0.5}
az.plot_hpd(X, lines.T,ax=ax,color='c',fill_kwargs=fill_kwargs,hdi_prob=0.89)

plt.scatter(df['in_std'],df['e_std'],color='crimson')

In [ ]:
#model 3 : multi-regression {in,out} --> energy

result = pd.DataFrame()
summary = pd.DataFrame()

x0 = df['out_std']
x1 = df['in_std']

model_3 = pm.Model()
with model_3:
    
    alpha = pm.Normal('alpha',0,1)
    beta = pm.Normal('beta',0,1,shape=2)
    obs_sigma = pm.Exponential('obs_sigma',1)
    
    reg = alpha + beta[0] * x0 + beta[1] * x1
    
    lkh = pm.Normal('lkh',reg,obs_sigma,observed=df['e_std'])
    
    trace = pm.sample(500,tune=500)
    
    result = pm.trace_to_dataframe(trace)
    
    summary = az.summary(trace,hdi_prob=0.89)
    
    print (summary)
    az.plot_trace(trace)

In [ ]:
result.describe()

In [ ]:
rows = np.random.choice(result.index,replace=True,size=nr_rows)

lines = np.array([result.iloc[rows]['alpha'] + result.iloc[rows]['beta__0'] * X[i] \
                  + result.iloc[rows]['beta__1'] * X[i] for i in range(len(X))])

In [ ]:
summary

In [ ]:
_= plt.plot(X,lines,color='r',alpha=0.02)

ax = plt.gca()

label_1 = r'$\alpha$ : {:.2f} [HPDI : {:.2f} .. {:.2f}]'.format(summary.loc['alpha','mean'],
                                                                summary.loc['alpha','hdi_5.5%'],
                                                                summary.loc['alpha','hdi_94.5%'])

label_2 = r'$\beta_0$ : {:.2f} [HPDI : {:.2f} .. {:.2f}]'.format(summary.loc['beta[0]','mean'],
                                                              summary.loc['beta[0]','hdi_5.5%'],
                                                              summary.loc['beta[0]','hdi_94.5%'])

label_3 = r'$\beta_1$ : {:.2f} [HPDI : {:.2f} .. {:.2f}]'.format(summary.loc['beta[1]','mean'],
                                                              summary.loc['beta[1]','hdi_5.5%'],
                                                              summary.loc['beta[1]','hdi_94.5%'])


_= plt.plot(X,result['alpha'].mean() + result['beta__0'].mean() * X + result['beta__1'].mean() * X,
            color='k',ls='dashed',label=label_1 + '\n' + label_2 + '\n' + label_3)

plt.xlabel('in [std]')
plt.ylabel('energy [std]')

plt.legend(loc='upper left')

fill_kwargs = {'alpha': 0.5}
az.plot_hpd(X, lines.T,ax=ax,color='c',fill_kwargs=fill_kwargs,hdi_prob=0.89)

plt.scatter(df['in_std'],df['e_std'],color='crimson')
plt.scatter(df['out_std'],df['e_std'],color='green')

In [ ]:
df_all[['out_std','in_std','e_std']] = df_all[['out','in','e']].apply(standardize)
df_all


In [ ]:
# model 4 : two 'countries or houses with differing energy consumption, second has 10% more'

x0 = df_all['out_std'].values
x1 = df_all['in_std'].values 

country_idx = df_all['country_idx'].values

result = pd.DataFrame()
summary = pd.DataFrame()

model_4 = pm.Model()
with model_4:
    
    alpha = pm.Normal('alpha',0,10,shape=2)
    beta = pm.Normal('beta',0,10,shape=(2,2))
    obs_sigma = pm.Exponential('obs_sigma',1)

    reg = alpha[country_idx] + x0 * beta[0,country_idx] + x1 * beta[1,country_idx]

    lkh = pm.Normal('lkh',reg,obs_sigma,observed=df_all['e_std'])

    trace = pm.sample(500,tune=500)

    result = pm.trace_to_dataframe(trace)
    
    summary = az.summary(trace,hdi_prob=0.89)
    
    print (result.describe())
    print (summary)
    
    az.plot_trace(trace)

In [ ]:
rows = np.random.choice(result.index,replace=True,size=nr_rows)

lines_0 = np.array([result.iloc[rows]['alpha__0'] + result.iloc[rows]['beta__0_0'] * X[i] \
                  + result.iloc[rows]['beta__1_0'] * X[i] for i in range(len(X))])

lines_1 = np.array([result.iloc[rows]['alpha__1'] + result.iloc[rows]['beta__0_1'] * X[i] \
                  + result.iloc[rows]['beta__1_1'] * X[i] for i in range(len(X))])

In [ ]:
_= plt.plot(X,lines_0,color='r',alpha=0.02)

plt.plot(X,lines_1,color='g',alpha=0.02)


ax = plt.gca()


_= plt.plot(X,result['alpha__0'].mean() + result['beta__0_0'].mean() * X + result['beta__1_0'].mean() * X,
            color='k',ls='dashed')

_= plt.plot(X,result['alpha__1'].mean() + result['beta__0_1'].mean() * X + result['beta__1_1'].mean() * X,
            color='k',ls='dashed')


plt.xlabel('in [std]')
plt.ylabel('energy [std]')

plt.legend(loc='upper left')

fill_kwargs = {'alpha': 0.5}
az.plot_hpd(X, lines_0.T,ax=ax,color='c',fill_kwargs=fill_kwargs,hdi_prob=0.89)
az.plot_hpd(X, lines_1.T,ax=ax,color='lime',fill_kwargs=fill_kwargs,hdi_prob=0.89)



